### 행정안전부 보도자료 데이터 수집
- 정적페이지 데이터 수집 : html : bs, css-selector
- post방식
- 여러개의 페이지 데이터 수집

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 1. 웹 서비스 분석 : url

post방식일 떄에는 url이 바뀌지 않는다.  
payload 페이지에 있는 내용을 보면 된다.  
response를 보면 html코드가 있는것을 확인할 수 있는데 내가 원하는것이 있는지 preview를 통해 시각적으로 보자.

In [2]:
page = 3
url = "https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"

params = {
    "nttId": "0",
    "bbsTyCode": "BBST03",
    "bbsAttrbCode": "BBSA03",
    "authFlag": "Y",
    "pageIndex": page,
    "cal_url":" /sym/cal/EgovNormalCalPopup.do",
    "searchCnd": "0",
    "searchWrd": "",
}

### 2. request(url) > response : html

In [3]:
response = requests.post(url, params)
response

<Response [200]>

### 3. html(str) > bs_obj > bs_obj.select(css-selector) > text > DataFrame

In [4]:
dom = BeautifulSoup(response.text, 'html.parser')

In [7]:
# 게시글 리스트 데이터 선택 : 10개

elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")
len(elements)


10

In [ ]:
# 각 데이터에서 필요한 정보 수집

element.select("td")[1].select_one("a").get("href")  
여기에서 td의 2번째 값을 가져와라 -> 그다음 a tag태그 안에 있는 href 속성을 가져와라는 의미이다.

In [15]:
element = elements[0]

data = {
    "no" : element.select("td")[0].text.strip(),
    "title" : element.select("td")[1].text.strip(),
    "writer" : element.select("td")[2].text.strip(),
    "date" : element.select("td")[3].text.strip(),
    "pv" : element.select("td")[4].text.strip(),
    "link" : "https://www.mois.go.kr" + element.select("td")[1].select_one("a").get("href"),
}
data

{'no': '12931',
 'title': '사이버대학 성적·졸업증명서도 모바일 전자증명서로 발급한다',
 'writer': '',
 'date': '공공지능정책과',
 'pv': '2022.02.15.',
 'link': 'https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardArticle.do;jsessionid=iUVzcJQhyZ+3uk1aft+dJooP.node10?bbsId=BBSMSTR_000000000008&nttId=90455'}

In [17]:
# 데이터 프레임으로 만들기
# [ {row1}, {row2}]
datas = []
for element in elements:
    datas.append({
        "no" : element.select("td")[0].text.strip(),
        "title" : element.select("td")[1].text.strip(),
        "writer" : element.select("td")[2].text.strip(),
        "date" : element.select("td")[3].text.strip(),
        "pv" : element.select("td")[4].text.strip(),
        "link" : "https://www.mois.go.kr" + element.select("td")[1].select_one("a").get("href"),
    })
df = pd.DataFrame(datas)
df.tail()

,no,title,writer,date,pv,link
5,12926,"행정안전부 소통협력공간, 인구감소지역 최초 선정",,지역사회혁신정책과,2022.02.13.,https://www.mois.go.kr/frt/bbs/type010/commonS...
6,12925,국민이 제안하고 풀어가는 민생규제 혁신과제 공모전 개최,,지방규제혁신과,2022.02.13.,https://www.mois.go.kr/frt/bbs/type010/commonS...
7,12924,이제 스마트폰으로 주민등록증 확인 가능해진다... 4월 시범실시 예정,,주민과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...
8,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,,스마트행정기반과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...
9,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",,지방규제혁신과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...


### 4. function param : page

In [18]:
def mois(page):
    url = "https://www.mois.go.kr/frt/bbs/type010/commonSelectBoardList.do?bbsId=BBSMSTR_000000000008"

    params = {
        "nttId": "0",
        "bbsTyCode": "BBST03",
        "bbsAttrbCode": "BBSA03",
        "authFlag": "Y",
        "pageIndex": page,
        "cal_url":" /sym/cal/EgovNormalCalPopup.do",
        "searchCnd": "0",
        "searchWrd": "",
    }

    response = requests.post(url, params)
    dom = BeautifulSoup(response.text, 'html.parser')
    elements = dom.select("#print_area > div.table_wrap.type_01 > form > table > tbody > tr")

    datas = []
    for element in elements:
        datas.append({
            "no" : element.select("td")[0].text.strip(),
            "title" : element.select("td")[1].text.strip(),
            "writer" : element.select("td")[2].text.strip(),
            "date" : element.select("td")[3].text.strip(),
            "pv" : element.select("td")[4].text.strip(),
            "link" : "https://www.mois.go.kr" + element.select("td")[1].select_one("a").get("href"),
        })
    df = pd.DataFrame(datas)
    return df

In [19]:
mois(4)

,no,title,writer,date,pv,link
0,12921,데이터 기반 과학적 행정으로 디지털 대전환 시대 정부역량 강화,,공공데이터정책과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...
1,12920,2022년 지역균형 뉴딜 추진전략 논의,,지역균형뉴딜추진단 부단장 지역균형뉴딜팀,2022.02.09.,https://www.mois.go.kr/frt/bbs/type010/commonS...
2,12919,"정부, 2월 가뭄 예·경보 및 국가가뭄통계 발표",,기후재난대응과,2022.02.09.,https://www.mois.go.kr/frt/bbs/type010/commonS...
3,12918,디지털로 여는 좋은 세상을 실현할 정보기술(IT) 기업을 찾습니다,,디지털서비스정책과,2022.02.09.,https://www.mois.go.kr/frt/bbs/type010/commonS...
4,12917,지자체와 청년을 잇는 지역주도형 청년 일자리 2만 6천개 창출,,지역일자리경제과,2022.02.09.,https://www.mois.go.kr/frt/bbs/type010/commonS...
5,12916,"연1조 지방소멸대응기금, 인구감소지역에 집중 투자한다",,지역균형발전과,2022.02.08.,https://www.mois.go.kr/frt/bbs/type010/commonS...
6,12915,"주민이 직접 조례 제·개정 청구, 온라인으로 가능합니다",,선거의회자치법규과,2022.02.08.,https://www.mois.go.kr/frt/bbs/type010/commonS...
7,12914,"자치분권 2.0시대, 지방의회 역량강화를 위한 지방의정연수센터 개소",,지방의정연수센터,2022.02.08.,https://www.mois.go.kr/frt/bbs/type010/commonS...
8,12913,업무 처리 자동화(RPA)를 통한 디지털 전환 가속화,,협업정책과,2022.02.08.,https://www.mois.go.kr/frt/bbs/type010/commonS...
9,12912,2021년도 지방자치단체 혁신평가 결과 발표,,지역사회혁신정책과,2022.02.08.,https://www.mois.go.kr/frt/bbs/type010/commonS...


### 5. 여러 페이지 데이터 수집

-  데이터프레임을 하나로 합치기
- pd.concat

In [20]:
dfs = []
for page in range(1,4):
    print(page, end = " ")
    dfs.append(mois(page))


1 2 3 

In [25]:
result_df = pd.concat(dfs, ignore_index=True)
result_df.tail(5)

,no,title,writer,date,pv,link
25,12926,"행정안전부 소통협력공간, 인구감소지역 최초 선정",,지역사회혁신정책과,2022.02.13.,https://www.mois.go.kr/frt/bbs/type010/commonS...
26,12925,국민이 제안하고 풀어가는 민생규제 혁신과제 공모전 개최,,지방규제혁신과,2022.02.13.,https://www.mois.go.kr/frt/bbs/type010/commonS...
27,12924,이제 스마트폰으로 주민등록증 확인 가능해진다... 4월 시범실시 예정,,주민과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...
28,12923,스마트워크센터 밀집현상 해소를 위해 서울역 2호점 추가 개소,,스마트행정기반과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...
29,12922,"2022년 지방규제, 지역활력 제고에 초점 맞추기로",,지방규제혁신과,2022.02.10.,https://www.mois.go.kr/frt/bbs/type010/commonS...
